In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter
from sklearn import svm

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [ ]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('../../week0_network_analysis_project/')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
# # combine all json file in all-weeks8-9
# def slack_parser(path_channel):
#     """ parse slack data to extract useful informations from the json file
#         step of execution
#         1. Import the required modules
#         2. read all json file from the provided path
#         3. combine all json files in the provided path
#         4. extract all required informations from the slack data
#         5. convert to dataframe and merge all
#         6. reset the index and return dataframe
#     """

#     # specify path to get json files
#     combined = []
#     for json_file in glob.glob(f"{path_channel}*.json"):
#         with open(json_file, 'r', encoding="utf8") as slack_data:
#             combined.append(slack_data)

#     # loop through all json files and extract required informations
#     dflist = []
#     for slack_data in combined:

#         msg_type, msg_content, sender_id, time_msg, msg_dist, time_thread_st, reply_users, \
#         reply_count, reply_users_count, tm_thread_end = [],[],[],[],[],[],[],[],[],[]

#         for row in slack_data:
#             if 'bot_id' in row.keys():
#                 continue
#             else:
#                 msg_type.append(row['type'])
#                 msg_content.append(row['text'])
#                 if 'user_profile' in row.keys(): sender_id.append(row['user_profile']['real_name'])
#                 else: sender_id.append('Not provided')
#                 time_msg.append(row['ts'])
#                 if 'blocks' in row.keys() and len(row['blocks'][0]['elements'][0]['elements']) != 0 :
#                      msg_dist.append(row['blocks'][0]['elements'][0]['elements'][0]['type'])
#                 else: msg_dist.append('reshared')
#                 if 'thread_ts' in row.keys():
#                     time_thread_st.append(row['thread_ts'])
#                 else:
#                     time_thread_st.append(0)
#                 if 'reply_users' in row.keys(): reply_users.append(",".join(row['reply_users'])) 
#                 else:    reply_users.append(0)
#                 if 'reply_count' in row.keys():
#                     reply_count.append(row['reply_count'])
#                     reply_users_count.append(row['reply_users_count'])
#                     tm_thread_end.append(row['latest_reply'])
#                 else:
#                     reply_count.append(0)
#                     reply_users_count.append(0)
#                     tm_thread_end.append(0)
#         data = zip(msg_type, msg_content, sender_id, time_msg, msg_dist, time_thread_st,
#          reply_count, reply_users_count, reply_users, tm_thread_end)
#         columns = ['msg_type', 'msg_content', 'sender_name', 'msg_sent_time', 'msg_dist_type',
#          'time_thread_start', 'reply_count', 'reply_users_count', 'reply_users', 'tm_thread_end']

#         df = pd.DataFrame(data=data, columns=columns)
#         df = df[df['sender_name'] != 'Not provided']
#         dflist.append(df)

#     dfall = pd.concat(dflist, ignore_index=True)
#     dfall['channel'] = path_channel.split('/')[-1].split('.')[0]        
#     dfall = dfall.reset_index(drop=True)
    
#     return dfall


# def parse_slack_reaction(path, channel):
#     """get reactions"""
#     dfall_reaction = pd.DataFrame()
#     combined = []
#     for json_file in glob.glob(f"{path}*.json"):
#         with open(json_file, 'r') as slack_data:
#             combined.append(slack_data)

#     reaction_name, reaction_count, reaction_users, msg, user_id = [], [], [], [], []

#     for k in combined:
#         slack_data = json.load(open(k.name, 'r', encoding="utf-8"))
        
#         for i_count, i in enumerate(slack_data):
#             if 'reactions' in i.keys():
#                 for j in range(len(i['reactions'])):
#                     msg.append(i['text'])
#                     user_id.append(i['user'])
#                     reaction_name.append(i['reactions'][j]['name'])
#                     reaction_count.append(i['reactions'][j]['count'])
#                     reaction_users.append(",".join(i['reactions'][j]['users']))
                
#     data_reaction = zip(reaction_name, reaction_count, reaction_users, msg, user_id)
#     columns_reaction = ['reaction_name', 'reaction_count', 'reaction_users_count', 'message', 'user_id']
#     df_reaction = pd.DataFrame(data=data_reaction, columns=columns_reaction)
#     df_reaction['channel'] = channel
#     return df_reaction

# def get_community_participation(path):
#     """ specify path to get json files"""
#     combined = []
#     comm_dict = {}
#     for json_file in glob.glob(f"{path}*.json"):
#         with open(json_file, 'r') as slack_data:
#             combined.append(slack_data)
#     # print(f"Total json files is {len(combined)}")
#     for i in combined:
#         a = json.load(open(i.name, 'r', encoding='utf-8'))

#         for msg in a:
#             if 'replies' in msg.keys():
#                 for i in msg['replies']:
#                     comm_dict[i['user']] = comm_dict.get(i['user'], 0)+1
#     return comm_dict

In [ ]:
# def convert_2_timestamp(column, data):
#     """convert from unix time to readable timestamp
#         args: column: columns that needs to be converted to timestamp
#                 data: data that has the specified column
#     """
#     if column in data.columns.values:
#         timestamp_ = []
#         for time_unix in data[column]:
#             if time_unix == 0:
#                 timestamp_.append(0)
#             else:
#                 a = datetime.datetime.fromtimestamp(float(time_unix))
#                 timestamp_.append(a.strftime('%Y-%m-%d %H:%M:%S'))
#         return timestamp_
#     else: 
#         print(f"{column} not in data")

# def get_tagged_users(df):
#     """get all @ in the messages"""

#     return df['msg_content'].map(lambda x: re.findall(r'@U\w+', x))


    
# def map_userid_2_realname(user_profile: dict, comm_dict: dict, plot=False):
#     """
#     map slack_id to realnames
#     user_profile: a dictionary that contains users info such as real_names
#     comm_dict: a dictionary that contains slack_id and total_message sent by that slack_id
#     """
#     user_dict = {} # to store the id
#     real_name = [] # to store the real name
#     ac_comm_dict = {} # to store the mapping
#     count = 0
#     # collect all the real names
#     for i in range(len(user_profile['profile'])):
#         real_name.append(dict(user_profile['profile'])[i]['real_name'])

#     # loop the slack ids
#     for i in user_profile['id']:
#         user_dict[i] = real_name[count]
#         count += 1

#     # to store mapping
#     for i in comm_dict:
#         if i in user_dict:
#             ac_comm_dict[user_dict[i]] = comm_dict[i]

#     ac_comm_dict = pd.DataFrame(data= zip(ac_comm_dict.keys(), ac_comm_dict.values()),
#     columns=['LearnerName', '# of Msg sent in Threads']).sort_values(by='# of Msg sent in Threads', ascending=False)
    
#     if plot:
#         ac_comm_dict.plot.bar(figsize=(15, 7.5), x='LearnerName', y='# of Msg sent in Threads')
#         plt.title('Student based on Message sent in thread', size=20)
        
#     return ac_comm_dict

In [ ]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

### Minimum Essential To Do: Task 1

In [ ]:
# Who are the top and bottom 10  users by:
    # Reaction count?

import pandas as pd
from functools import reduce

data_path = os.path.join(rpath, "data/anonymized")
data_loader = SlackDataLoader(data_path)
directories = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
channel_name = 'all-week8'

dataframe_by_directory = [
    data_loader.parse_slack_reaction(os.path.join(data_path, directory), channel_name).assign(directory=directory)
    for directory in directories
]

# Concatenate all DataFrames into a single DataFrame
df_combined = reduce(lambda df1, df2: pd.concat([df1, df2], ignore_index=True), dataframe_by_directory)


reaction_df = pd.concat(dataframe_by_directory, ignore_index=True)

users_df= pd.DataFrame(data_loader.get_users())
users_df.rename(columns={'id':'user_id'}, inplace = True)

reactions_merged = pd.merge(reaction_df, users_df, on='user_id', how='outer')


top_10_reactions = reactions_merged.groupby('real_name')['reaction_count'].sum().reset_index(name='total_reaction_count').sort_values(by='total_reaction_count', ascending=False).head(10)

bottom_10_reactions = reactions_merged.groupby('real_name')['reaction_count'].sum().reset_index(name='total_reaction_count').sort_values(by='total_reaction_count', ascending=True).head(10)

print(top_10_reactions)
print(bottom_10_reactions)


In [ ]:
# Who are the top and bottom 10  users by:
        # Reply count? 
        # Mention? 
        # Message count? 

data_path = os.path.join(rpath, "data/anonymized")
data_loader = SlackDataLoader(data_directory)
folders = [d for d in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, d))]

dataframes_by_folder = [data_loader.slack_parser(os.path.join(data_path, folder)).assign(folder=folder) for folder in folders]

parsed_data = pd.concat(dataframes_by_folder, ignore_index=True)
parsed_data.head(5)

reply_count_by_message = parsed_data.groupby('msg_content')['reply_count'].sum().reset_index(name='Reply count')
print(reply_count_by_message)

message_count_by_user = parsed_data.groupby('sender_name').size().reset_index(name='Message count')

mention_count_by_user = parsed_data['msg_content'].str.count('@\w+').groupby(parsed_data['sender_name']).sum().reset_index(name='Mention count')

reply_count_by_user = parsed_data.groupby('sender_name')['reply_count'].sum().reset_index(name='Reply count')

user_stats = pd.merge(message_count_by_user, mention_count_by_user, on='sender_name', how='outer')
user_stats = pd.merge(user_stats, reply_count_by_user, on='sender_name', how='outer')

user_stats = user_stats.fillna(0)

user_stats['Total count'] = user_stats['Message count'] + user_stats['Mention count'] + user_stats['Reply count']

top_10_reply = user_stats.sort_values(by='Reply count', ascending=False).head(10)
bottom_10_reply = user_stats.sort_values(by='Reply count', ascending=True).head(10)

top_10_mention = user_stats.sort_values(by='Mention count', ascending=False).head(10)
bottom_10_mention = user_stats.sort_values(by='Mention count', ascending=True).head(10)

top_10_message = user_stats.sort_values(by='Message count', ascending=False).head(10)
bottom_10_message = user_stats.sort_values(by='Message count', ascending=True).head(10)

print("Top 10 Users by Reply Count:")
print(top_10_reply[['sender_name', 'Reply count']])

print("\nBottom 10 Users by Reply Count:")
print(bottom_10_reply[['sender_name', 'Reply count']])

print("\nTop 10 Users by Mention Count:")
print(top_10_mention[['sender_name', 'Mention count']])

print("\nBottom 10 Users by Mention Count:")
print(bottom_10_mention[['sender_name', 'Mention count']])

print("\nTop 10 Users by Message Count:")
print(top_10_message[['sender_name', 'Message count']])

print("\nBottom 10 Users by Message Count:")
print(bottom_10_message[['sender_name', 'Message count']])




In [14]:
#What are the top 10 messages by 
    # Replies?
    # Reactions?
    # Mentions?
data_path = os.path.join(rpath, "data/anonymized")
data_loader = SlackDataLoader(data_path)
folders = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]

dataframes_by_folder = [data_loader.slack_parser(os.path.join(data_path, folder)).assign(folder=folder) for folder in folders]

parsed_data = pd.concat(dataframes_by_folder, ignore_index=True)
parsed_data.head(5)

reply_count_by_message = parsed_data.groupby('msg_content')['reply_count'].sum().reset_index(name='Reply count')

reaction_count_by_message = parsed_data.groupby('msg_content')['reply_users_count'].sum().reset_index(name='Reaction count')

mention_count_by_message = parsed_data['msg_content'].str.count('@\w+').groupby(parsed_data['msg_content']).sum().reset_index(name='Mention count')

message_stats = pd.merge(reply_count_by_message, reaction_count_by_message, on='msg_content', how='outer')
message_stats = pd.merge(message_stats, mention_count_by_message, on='msg_content', how='outer')

message_stats = message_stats.fillna(0)

message_stats['Total count'] = message_stats['Reply count'] + message_stats['Reaction count'] + message_stats['Mention count']

top_10_reply_message = message_stats.sort_values(by='Reply count', ascending=False).head(10)
top_10_reaction_message = message_stats.sort_values(by='Reaction count', ascending=False).head(10)
top_10_mention_message = message_stats.sort_values(by='Mention count', ascending=False).head(10)

print("\nTop 10 Messages by Reply Count:")
print(top_10_reply_message[['msg_content', 'Reply count']])

print("\nTop 10 Messages by Reaction Count:")
print(top_10_reaction_message[['msg_content', 'Reaction count']])

print("\nTop 10 Messages by Mention Count:")
print(top_10_mention_message[['msg_content', 'Mention count']])



Error loading channels: Expecting value: line 1 column 1 (char 0)

Top 10 Messages by Reply Count:
                                             msg_content  Reply count
2304           <@u03v1am5tfa> how many crushes you have?           75
6051   first impression: people think i am older than...           63
476    1. how do i find my jupyter notebook file in m...           61
7475   how many here follow football so much? i think...           57
5915                     ethiopian new year loading ....           54
6775   hello\n1. <@u03uvhcv6kb>\n2. <@u03ug5vfn03>\n3...           45
2151   <@u03uur571a5> solutions to funding account an...           44
13902  we had a plan to meet today 2:00 eat at friend...           36
120    *<!here> reminder*:timer_clock:\n\n*please not...           33
12960  the debate was great. we <@u03u1j51vfz>, <@u03...           31

Top 10 Messages by Reaction Count:
                                             msg_content  Reaction count
12960  the debate was 

In [ ]:
# Which channel has the highest activity? 
# Which channel appears at the right top corner when you plot a 2D scatter plot where x-axis is the number of messages in the channel,
# y-axis is the sum of number of replies and reactions, and the color representing channels?



import matplotlib.pyplot as plt
import seaborn as sns

data_path = os.path.join(rpath, "data/anonymized")
data_loader = SlackDataLoader(data_path)
folders = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]

dataframes_by_folder = [data_loader.slack_parser(os.path.join(data_path, folder)).assign(folder=folder) for folder in folders]

parsed_data = pd.concat(dataframes_by_folder, ignore_index=True)
parsed_data.head(5)

message_count_by_channel = parsed_data.groupby('channel')['msg_content'].count().reset_index(name='Message count')
activity_by_channel = parsed_data.groupby('channel')[['reply_count', 'reply_users_count']].sum().reset_index()
activity_by_channel['Total activity'] = activity_by_channel['reply_count'] + activity_by_channel['reply_users_count']
channel_stats = pd.merge(message_count_by_channel, activity_by_channel, on='channel', how='outer')
channel_stats = channel_stats.fillna(0)

plt.figure(figsize=(12, 8))
scatter_plot = sns.scatterplot(x='Message count', y='Total activity', hue='channel', data=channel_stats, s=100)
right_top_channel = channel_stats.loc[channel_stats['Total activity'].idxmax()]
scatter_plot.annotate(right_top_channel['channel'], 
                      (right_top_channel['Message count'], right_top_channel['Total activity']),
                      textcoords="offset points", 
                      xytext=(-10, 5),
                      ha='right')
plt.show()

most_active_channel = channel_stats.loc[channel_stats['Total activity'].idxmax()]
print(f"The channel with the highest activity is: {most_active_channel['channel']}")



In [ ]:
# What fraction of messages are replied within the first 5mins?
# Plot a 2D scatter plot such that x-axis is the time difference between the message timestamp and the first reply message, 
# y-axis is the time of the day (in 24hr format), color representing channels? 


import matplotlib.pyplot as plt
import seaborn as sns

data_path = os.path.join(rpath, "data/anonymized")
data_loader = SlackDataLoader(data_path)
folders = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]

dataframes_by_folder = [data_loader.slack_parser(os.path.join(data_path, folder)).assign(folder=folder) for folder in folders]

parsed_data = pd.concat(dataframes_by_folder, ignore_index=True)
parsed_data.head(5)
# slack_parser_df.head(5)

parsed_data['time_thread_start'] = pd.to_numeric(parsed_data['time_thread_start'], errors='coerce')
parsed_data['msg_sent_time'] = pd.to_numeric(parsed_data['msg_sent_time'], errors='coerce')

parsed_data['time_diff_first_reply'] = (parsed_data['time_thread_start'] - parsed_data['msg_sent_time']) / 60

parsed_data['time_of_day'] = pd.to_datetime(parsed_data['msg_sent_time'], unit='s').dt.strftime('%H:%M')

scatter_data = parsed_data[['time_diff_first_reply', 'time_of_day', 'channel']]

# Plot a 2D scatter plot
plt.figure(figsize=(12, 8))
scatter_plot = sns.scatterplot(x='time_diff_first_reply', y='time_of_day', hue='channel', data=scatter_data, s=100)

# Display the plot
plt.show()

### Minimum Essential To Do: Task 2

In [ ]:
# # Test for slack_parser method
# import unittest
# test_path = os.path.join(rpath, "tests")

# # Load the test suite
# loader = unittest.TestLoader()
# suite = loader.discover('path/to/tests', pattern='test_*.py')

# # Run the tests
# runner = unittest.TextTestRunner()
# result = runner.run(suite)

# # Check the test result
# result.wasSuccessful()